# Imports

In [ ]:
!pip install irrCAC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from sklearn.metrics import cohen_kappa_score, f1_score
import seaborn as sns
from statsmodels.stats.inter_rater import fleiss_kappa
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
import itertools

In [ ]:
import irrCAC
from irrCAC.datasets import raw_4raters
from irrCAC.raw import CAC

# Merging files
The annotators datasets with agreement were merged together into topic x platform files and then into full corpus.


In [ ]:
path = '/content/drive/MyDrive/MA thesis/2nd round/'

## Topic x platform

### Reddit climate

In [ ]:
file_paths = [
    path + 'Reddit_r2_a1_zgodność_climate.xlsx',
    path + 'Reddit_r2_a2_zgodność_climate.xlsx',
    path + 'Reddit_r2_a3_zgodność_climate.xlsx',
    path + 'Reddit_r2_a4_zgodność_climate.xlsx'
]

annotators = ["bs", "fs", "ir", "wl"]
platform = "Reddit"
topic = "CC"
bias_columns = ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']

dataframes = [pd.read_excel(file) for file in file_paths]

def merge_files_with_annotators(dataframes, annotators, bias_columns, platform, topic):
    merged_df = dataframes[0][['title', 'post_id_x', 'comment_id', 'comment_author', 'text']].copy()

    merged_df['platform'] = platform
    merged_df['topic'] = topic

    for df, annotator in zip(dataframes, annotators):
        annotated_cols = {col: f"{col}_{annotator}" for col in bias_columns}
        annotator_df = df[['title', 'post_id_x', 'comment_id', 'comment_author', 'text'] + bias_columns].rename(columns=annotated_cols)
        annotator_df['platform'] = platform
        annotator_df['topic'] = topic
        merged_df = pd.merge(merged_df, annotator_df, on=['title', 'post_id_x', 'comment_id', 'comment_author', 'text', 'platform', 'topic'], how='outer')

    return merged_df

In [ ]:
merged_df = merge_files_with_annotators(dataframes, annotators, bias_columns, platform, topic)

merged_df

,title,post_id_x,comment_id,comment_author,text,platform,topic,affect_h_bs,availability_h_bs,representativeness_h_bs,...,affect_h_ir,availability_h_ir,representativeness_h_ir,confirmation_b_ir,all_or_nothing_ir,affect_h_wl,availability_h_wl,representativeness_h_wl,confirmation_b_wl,all_or_nothing_wl
0,"""Our findings predict that a temperature incre...",1atygy6,NaN,NaN,0,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Another warning from an insurance company,1asczho,NaN,NaN,0,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Best estimated places to move to in North Amer...,1atzufx,NaN,NaN,"Sorry for the long title haha, just wondering ...",Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Biofuels are a problem,1asju6i,NaN,NaN,Destroying rain forest to grow sugarcane for b...,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Books on climate change?,1asy26j,NaN,NaN,Hello! What's your recommendation for books th...,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,NaN,1aup2v9,krbl5ye,Gullible-Minute-9482,I was thinking the same thing in regard to you...,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,NaN,1aup2v9,krdbxx5,DocQuang,Carbon emissions is a subset of environmental ...,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,NaN,1aup2v9,krdhy8z,No_Job_5208,Fair enough if that's what YOU believe!..,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090,NaN,1aup2v9,krf5n9i,No_Job_5208,But with every tonne of mined lithium 15 tonne...,Reddit,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df.to_excel('/content/drive/MyDrive/MA thesis/Results/merged_reddit_cc_with_platform_topic.xlsx', index=False)

### Reddit AI

In [ ]:
file_paths = [
    path + 'Reddit_r2_a1_zgodność_AI.xlsx',
    path + 'Reddit_r2_a2_zgodność_AI.xlsx',
    path + 'Reddit_r2_a3_zgodność_AI.xlsx',
    path + 'Reddit_r2_a4_zgodność_AI.xlsx'
]

platform = "Reddit"
topic = "AI"

dataframes = [pd.read_excel(file) for file in file_paths]

In [ ]:
merged_df = merge_files_with_annotators(dataframes, annotators, bias_columns, platform, topic)

merged_df

,title,post_id_x,comment_id,comment_author,text,platform,topic,affect_h_bs,availability_h_bs,representativeness_h_bs,...,affect_h_ir,availability_h_ir,representativeness_h_ir,confirmation_b_ir,all_or_nothing_ir,affect_h_wl,availability_h_wl,representativeness_h_wl,confirmation_b_wl,all_or_nothing_wl
0,AI Just Cleared A Big Hurdle On The Road To Nu...,1b0lcym,NaN,NaN,0,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AI and the Future,1b0s04s,NaN,NaN,Truck Drivers\nFast-Food Workers\nManufacturin...,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,Accelerating AI for Unified Economic Relief: A...,1b0nw0t,NaN,NaN,Lately I’ve been reflecting on the potential s...,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,After building with the OpenAI API,1b0zjt5,NaN,NaN,0,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Can we ban posts with personal fantasies? This...,1b0hq9p,NaN,NaN,\nThe title.,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,NaN,1b0zjt5,kscvdv6,HalfSecondWoe,"All of those are perfectly fair points, but so...",Reddit,AI,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,NaN,1b0zjt5,kscxn8k,TheMysteryCheese,"Yes agreed, I'm just feeling like GPT-4 is mor...",Reddit,AI,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,NaN,1b0zjt5,ksdhl29,MayorOfMeridium,Low IQ: GPT-3.5 is all you need.\n\nHigh IQ: G...,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1090,NaN,1b0zjt5,ksew5db,Longjumping-Bake-557,I find myself getting angry at 3.5 for meander...,Reddit,AI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df.to_excel('/content/drive/MyDrive/MA thesis/Results/merged_reddit_ai_with_platform_topic.xlsx', index=False)

### X climate

In [ ]:
file_paths = [
    path + 'Twitter_r2_a1_zgodność_climate.xlsx',
    path + 'Twitter_r2_a2_zgodność_climate.xlsx',
    path + 'Twitter_r2_a3_zgodność_climate.xlsx',
    path + 'Twitter_r2_a4_zgodność_climate.xlsx'
]

# Load each file into a DataFrame
dataframes = [pd.read_excel(file) for file in file_paths]

In [ ]:
platform = "Twitter"
topic = "CC"
merged_twitter_cc = merge_files_with_annotators(dataframes, annotators, bias_columns, platform, topic)

merged_twitter_cc

,Conversation ID,Tweet ID,Author ID,Created At,text,platform,topic,affect_h_bs,availability_h_bs,representativeness_h_bs,...,affect_h_ir,availability_h_ir,representativeness_h_ir,confirmation_b_ir,all_or_nothing_ir,affect_h_wl,availability_h_wl,representativeness_h_wl,confirmation_b_wl,all_or_nothing_wl
0,'1763955890758304100,'1763955890758304100,'16106584,2024-03-02 15:53:51,"Scotland, Ireland, Denmark, Finland and Norway...",Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'1763955890758304100,'1763968495081918910,'1441765045885550610,2024-03-02 16:43:56,@stillgray Sometimes I forget that Ian is a ch...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,'1763955890758304100,'1763969263780667579,'1685778115702992896,2024-03-02 16:47:00,@stillgray Insanity. Basically the only way fo...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,'1763955890758304100,'1763970413049368672,'1514255325314138119,2024-03-02 16:51:34,@stillgray REPLACEMENT THEORY! It’s here.,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'1763955890758304100,'1763970691693679006,'1594279583100157952,2024-03-02 16:52:40,@stillgray Pathetic. They are not Europeans. T...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,'1766405620737904651,'1766427401234301115,'1439048046453657605,2024-03-09 11:34:45,@rhosking252 You can add to the list the Wall ...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452,'1766405620737904651,'1766442320570761513,'1511793748518264842,2024-03-09 12:34:02,@liam081963 @rhosking252 and there lies yet an...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453,'1766405620737904651,'1766446078386225631,'1749914808810414080,2024-03-09 12:48:58,@rhosking252 On a good note our planet will ne...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
454,'1766405620737904651,'1766446306325631180,'1749914808810414080,2024-03-09 12:49:53,@rhosking252 To force change in some way then ...,Twitter,CC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_twitter_cc.to_excel('/content/drive/MyDrive/MA thesis/Results/merged_twitter_cc_with_platform_topic.xlsx', index=False)

### X AI

In [ ]:
file_paths = [
    path + 'Twitter_r2_a1_zgodność_AI.xlsx',
    path + 'Twitter_r2_a2_zgodność_AI.xlsx',
    path + 'Twitter_r2_a3_zgodność_AI.xlsx',
    path + 'Twitter_r2_a4_zgodność_AI.xlsx'
]

# Load each file into a DataFrame
dataframes = [pd.read_excel(file) for file in file_paths]

In [ ]:
platform = "Twitter"
topic = "AI"
merged_twitter_ai = merge_files_with_annotators(dataframes, annotators, bias_columns, platform, topic)

In [ ]:
merged_twitter_ai.to_excel('/content/drive/MyDrive/MA thesis/Results/merged_twitter_ai_with_platform_topic.xlsx', index=False)

## Topic and platform separately


In [ ]:
path = "/content/drive/MyDrive/MA thesis/Results/"
merged_file_paths = [
    path + 'merged_reddit_cc_with_platform_topic.xlsx',
    path + 'merged_reddit_ai_with_platform_topic.xlsx',
    path + 'merged_twitter_cc_with_platform_topic.xlsx',
    path + 'merged_twitter_ai_with_platform_topic.xlsx'
]

cc_files = [
    path + 'merged_reddit_cc_with_platform_topic.xlsx',
    path + 'merged_twitter_cc_with_platform_topic.xlsx'
]
ai_files = [
    path + 'merged_reddit_ai_with_platform_topic.xlsx',
    path + 'merged_twitter_ai_with_platform_topic.xlsx'
]


def load_and_concat_files(file_paths):
    dataframes = [pd.read_excel(file) for file in file_paths]
    return pd.concat(dataframes, ignore_index=True)

# Merge Reddit files
reddit_files = merged_file_paths[:2]
reddit_df = load_and_concat_files(reddit_files)
reddit_output_path = path + 'merged_reddit_r2.xlsx'
reddit_df.to_excel(reddit_output_path, index=False)

# Merge Twitter files
twitter_files = merged_file_paths[2:]
twitter_df = load_and_concat_files(twitter_files)
twitter_output_path = path + 'merged_twitter_r2.xlsx'
twitter_df.to_excel(twitter_output_path, index=False)

# Merge CC files (Reddit + Twitter CC)
cc_df = load_and_concat_files(cc_files)
cc_output_path = path + 'merged_cc_r2.xlsx'
cc_df.to_excel(cc_output_path, index=False)

# Merge AI files (Reddit + Twitter AI)
ai_df = load_and_concat_files(ai_files)
ai_output_path = path + 'merged_ai_r2.xlsx'
ai_df.to_excel(ai_output_path, index=False)

# Summary statistics
print("Reddit Rows:", len(reddit_df))
print("Twitter Rows:", len(twitter_df))
print("CC Rows:", len(cc_df))
print("AI Rows:", len(ai_df))

Reddit Rows: 2184
Twitter Rows: 815
CC Rows: 1548
AI Rows: 1451


## Merge full corpus

In [ ]:
# Define the paths for the merged files
path = "/content/drive/MyDrive/MA thesis/Results/"
merged_file_paths = [
    path + 'merged_reddit_cc_with_platform_topic.xlsx',
    path + 'merged_reddit_ai_with_platform_topic.xlsx',
    path + 'merged_twitter_cc_with_platform_topic.xlsx',
    path + 'merged_twitter_ai_with_platform_topic.xlsx'
]

# Load each merged file, keeping only the required columns
columns_to_keep = ['title', 'text', 'platform', 'topic']
annotation_prefixes = ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']

# Load each file, ensuring only required columns are included
merged_dataframes = []
for file in merged_file_paths:
    df = pd.read_excel(file)
    # Keep title, text, platform, topic, and annotation columns
    cols = [col for col in df.columns if col in columns_to_keep or col.startswith(tuple(annotation_prefixes))]
    df = df[cols]
    # Replace NaN with 0
    df.fillna(0, inplace=True)
    merged_dataframes.append(df)

# Concatenate all DataFrames
final_merged_df = pd.concat(merged_dataframes, ignore_index=True)

# Summary statistics to verify consistency
row_counts = [len(df) for df in merged_dataframes]
total_original_rows = sum(row_counts)
total_merged_rows = len(final_merged_df)

print("Summary Statistics:")
print(f"Total rows in original files: {total_original_rows}")
print(f"Total rows in merged file: {total_merged_rows}")

# Check if the row counts match
if total_merged_rows == total_original_rows:
    print("All rows are accounted for in the merged file.")
else:
    print("Warning: There is a discrepancy in row counts.")

# Display the final merged DataFrame for verification
print(final_merged_df.head())

Summary Statistics:
Total rows in original files: 3635
Total rows in merged file: 3635
All rows are accounted for in the merged file.
                                               title  \
0  "Our findings predict that a temperature incre...   
1          Another warning from an insurance company   
2  Best estimated places to move to in North Amer...   
3                             Biofuels are a problem   
4                           Books on climate change?   

                                                text platform topic  \
0                                                  0   Reddit    CC   
1                                                  0   Reddit    CC   
2  Sorry for the long title haha, just wondering ...   Reddit    CC   
3  Destroying rain forest to grow sugarcane for b...   Reddit    CC   
4  Hello! What's your recommendation for books th...   Reddit    CC   

   affect_h_bs  availability_h_bs  representativeness_h_bs  confirmation_b_bs  \
0          0.0       

In [ ]:
# Save the final merged DataFrame to a file
final_merged_file = path + 'merged_r2_topics_platform.xlsx'
final_merged_df.to_excel(final_merged_file, index=False)

# Saving files

## Main_Agree_All_HAnn

In [ ]:
# merged_r2 = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_r2_topics_platform.xlsx')

r2_reddit_cc = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_reddit_cc_with_platform_topic.xlsx')
r2_reddit_ai = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_reddit_ai_with_platform_topic.xlsx')
r2_twitter_cc = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_twitter_cc_with_platform_topic.xlsx')
r2_twitter_ai = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_twitter_ai_with_platform_topic.xlsx')

r2_twitter = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_twitter_r2.xlsx')
r2_reddit = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/merged_reddit_r2.xlsx')

r2_cc = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/Iteration 2/merged_cc_r2.xlsx')
r2_ai = pd.read_excel('/content/drive/MyDrive/MA thesis/Results/Iteration 2/merged_ai_r2.xlsx')

In [ ]:
agreement_df = pd.concat([r2_cc, r2_ai], ignore_index=True)

In [ ]:
agreement_df.to_excel('/content/drive/MyDrive/MA thesis/Results/agreement_df_r2.xlsx', index=False)

## Main_All_HAnn


In [ ]:
path = '/content/drive/MyDrive/MA thesis/2nd round/Whole dataset/Platform x Topic files/'

twitter_file_paths = [
    path + 'Twitter_Round2_Annotator_1_with_topic.xlsx',
    path + 'Twitter_Round2_Annotator_2_with_topic.xlsx',
    path + 'Twitter_Round2_Annotator_3_with_topic.xlsx',
    path + 'Twitter_Round2_Annotator_4_with_topic.xlsx'
]

reddit_file_paths = [
    path + 'Reddit_Round2_Annotator_1_with_topic.xlsx',
    path + 'Reddit_Round2_Annotator_2_with_topic.xlsx',
    path + 'Reddit_Round2_Annotator_3_with_topic.xlsx',
    path + 'Reddit_Round2_Annotator_4_with_topic.xlsx'
]

In [ ]:
annotators = ["bs", "fs", "ir", "wl"]
bias_columns = ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']

def merge_files_with_annotators(dataframes, annotators, bias_columns, merge_keys):
    merged_df = dataframes[0][merge_keys].copy()

    for df, annotator in zip(dataframes, annotators):
        annotated_cols = {col: f"{col}_{annotator}" for col in bias_columns}
        annotator_df = df[merge_keys + bias_columns].rename(columns=annotated_cols)
        merged_df = pd.merge(merged_df, annotator_df, on=merge_keys, how='outer')

    return merged_df

# Processing Twitter files
twitter_dfs = []
for fp in twitter_file_paths:
    df = pd.read_excel(fp)
    df.rename(columns={"Text": "text"}, inplace=True)
    twitter_dfs.append(df)

twitter_merge_keys = ['Conversation ID', 'Tweet ID', 'Author ID', 'Created At', 'text', 'platform', 'topic']

twitter_merged = merge_files_with_annotators(
    dataframes=twitter_dfs,
    annotators=annotators,
    bias_columns=bias_columns,
    merge_keys=twitter_merge_keys
)

# Processing Reddit files
reddit_dfs = []
for fp in reddit_file_paths:
    df = pd.read_excel(fp)

    # Helper function to combine title and text.
    def combine_title_text(row):
        title = row['title']
        text = row['text']
        if text in [0, "0"]:
            text = ""
        if pd.isna(title):
            return text
        if text:
            return f"{title} {text}"
        else:
            return title

    df['text'] = df.apply(combine_title_text, axis=1)
    df.drop(columns=['title'], inplace=True)
    reddit_dfs.append(df)

reddit_merge_keys = ['post_id_x', 'comment_id', 'comment_author', 'text', 'platform', 'topic']

reddit_merged = merge_files_with_annotators(
    dataframes=reddit_dfs,
    annotators=annotators,
    bias_columns=bias_columns,
    merge_keys=reddit_merge_keys
)

# Merging both platforms
final_df = pd.concat([twitter_merged, reddit_merged], ignore_index=True, sort=False)

In [ ]:
final_df.to_excel("/content/drive/MyDrive/MA thesis/2nd round/Whole dataset/Platform x Topic files/merged_whole_with_topic.xlsx", index=False)

# Descriptive stats


## Main_Agree_All_HAnn
agreement_df_r2.xlsx is called Main_Agree_All_HAnn in the files

In [ ]:
bias_columns = [
    "affect_h_bs", "availability_h_bs", "representativeness_h_bs", "confirmation_b_bs", "all_or_nothing_bs",
    "affect_h_fs", "availability_h_fs", "representativeness_h_fs", "confirmation_b_fs", "all_or_nothing_fs",
    "affect_h_ir", "availability_h_ir", "representativeness_h_ir", "confirmation_b_ir", "all_or_nothing_ir",
    "affect_h_wl", "availability_h_wl", "representativeness_h_wl", "confirmation_b_wl", "all_or_nothing_wl",
]

df_long = agreement_df.melt(
    id_vars=[],
    value_vars=bias_columns,
    var_name="bias_annotator",
    value_name="value"
)

df_long = df_long[df_long["value"] == 1]

df_long["annotator"] = df_long["bias_annotator"].str.split("_").str[-1]
df_long["bias_type"] = df_long["bias_annotator"].str.split("_").str[0]

grouped = df_long.groupby(["annotator", "bias_type"]).size().reset_index(name="count")

total_per = grouped.groupby("annotator")["count"].sum().reset_index(name="total")
grouped = grouped.merge(total_per, on="annotator")

grouped["percent"] = (grouped["count"] / grouped["total"] * 100).round(2)

bias_map = {
    "affect": "Affect heuristic",
    "availability": "Availability heuristic",
    "representativeness": "Representativeness heuristic",
    "confirmation": "Confirmation bias",
    "all": "All-or-nothing thinking"
}
grouped["bias_full"] = grouped["bias_type"].map(bias_map)

order = ["Affect heuristic", "Availability heuristic", "Representativeness heuristic", "Confirmation bias", "All-or-nothing thinking"]
grouped["bias_full"] = pd.Categorical(grouped["bias_full"], categories=order, ordered=True)
grouped = grouped.sort_values(["annotator", "bias_full"])

In [ ]:
grouped[["annotator","bias_full","count","percent"]]

,annotator,bias_full,count,percent
0,bs,Affect heuristic,25,22.94
2,bs,Availability heuristic,15,13.76
4,bs,Representativeness heuristic,8,7.34
3,bs,Confirmation bias,33,30.28
1,bs,All-or-nothing thinking,28,25.69
5,fs,Affect heuristic,594,62.86
7,fs,Availability heuristic,121,12.80
9,fs,Representativeness heuristic,15,1.59
8,fs,Confirmation bias,199,21.06
6,fs,All-or-nothing thinking,16,1.69


In [ ]:
bias_columns = [
    "affect_h_bs", "availability_h_bs", "representativeness_h_bs", "confirmation_b_bs", "all_or_nothing_bs",
    "affect_h_fs", "availability_h_fs", "representativeness_h_fs", "confirmation_b_fs", "all_or_nothing_fs",
    "affect_h_ir", "availability_h_ir", "representativeness_h_ir", "confirmation_b_ir", "all_or_nothing_ir",
    "affect_h_wl", "availability_h_wl", "representativeness_h_wl", "confirmation_b_wl", "all_or_nothing_wl",
]

df_long = agreement_df.melt(
    id_vars=["topic", "platform"],
    value_vars=bias_columns,
    var_name="bias_annotator",
    value_name="value"
)

df_long["annotator"] = df_long["bias_annotator"].str.split("_").str[-1]  # Extract last part (e.g., bs, fs, etc.)
df_long["bias_type"] = df_long["bias_annotator"].str.split("_").str[0]   # Extract first part (e.g., affect, availability, etc.)

df_long = df_long.dropna(subset=["value"])
df_long_filtered = df_long[df_long["value"] == 1]
grouped_stats = df_long_filtered.groupby(["annotator", "topic", "platform", "bias_type"]).size().reset_index(name="count")
total_instances = df_long.shape[0]
grouped_stats["percentage"] = (grouped_stats["count"] / total_instances) * 100

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
grouped_stats

,annotator,topic,platform,bias_type,count,percentage
0,bs,AI,Reddit,affect,11,0.652819
1,bs,AI,Reddit,all,19,1.127596
2,bs,AI,Reddit,representativeness,6,0.356083
3,bs,AI,Twitter,affect,1,0.059347
4,bs,AI,Twitter,availability,1,0.059347
5,bs,AI,Twitter,representativeness,1,0.059347
6,bs,CC,Reddit,affect,13,0.771513
7,bs,CC,Reddit,all,6,0.356083
8,bs,CC,Reddit,availability,14,0.830861
9,bs,CC,Reddit,confirmation,33,1.958457


In [ ]:
topic_counts = df_long_filtered.groupby(["topic", "bias_type"]).size().reset_index(name="count")

In [ ]:
topic_counts

,topic,bias_type,count
0,AI,affect,917
1,AI,all,162
2,AI,availability,207
3,AI,confirmation,280
4,AI,representativeness,84
5,CC,affect,173
6,CC,all,32
7,CC,availability,99
8,CC,confirmation,158
9,CC,representativeness,31


In [ ]:
topic_annotator_counts = df_long_filtered.groupby(["topic", "annotator"]).size().reset_index(name="count")
topic_annotator_counts

,topic,annotator,count
0,AI,bs,75
1,AI,fs,894
2,AI,ir,643
3,AI,wl,38
4,CC,bs,66
5,CC,fs,276
6,CC,ir,120
7,CC,wl,31


In [ ]:
platform_annotator_counts = df_long_filtered.groupby(["platform", "annotator"]).size().reset_index(name="count")
platform_annotator_counts

,platform,annotator,count
0,Reddit,bs,138
1,Reddit,fs,1100
2,Reddit,ir,724
3,Reddit,wl,57
4,Twitter,bs,3
5,Twitter,fs,70
6,Twitter,ir,39
7,Twitter,wl,12


## Main_All_HAnn
merged_whole_with_topic.xlsx is later called Main_All_HAnn_with_topics.xlxs in the files

In [ ]:
whole_dataset = pd.read_excel('/content/drive/MyDrive/MA thesis/2nd round/Whole dataset/merged_whole_with_topic.xlsx')

In [ ]:
bias_bases = ["affect_h", "availability_h", "representativeness_h", "confirmation_b", "all_or_nothing"]
annotators = ["bs", "fs", "ir", "wl"]

bias_columns = [f"{base}_{annotator}" for annotator in annotators for base in bias_bases]

available_bias_columns = [col for col in bias_columns if col in whole_dataset.columns]

df_long = whole_dataset.melt(
    id_vars=["topic", "platform"],
    value_vars=available_bias_columns,
    var_name="bias_annotator",
    value_name="value"
)

df_long[['bias', 'annotator']] = df_long["bias_annotator"].str.rsplit('_', n=1, expand=True)
df_long = df_long.dropna(subset=["value"])
df_long_filtered = df_long[df_long["value"] == 1]

grouped_stats = df_long_filtered.groupby(
    ["topic", "platform", "annotator", "bias"]
).size().reset_index(name="count")

grouped_stats["total_count"] = grouped_stats.groupby(
    ["topic", "platform", "annotator"]
)["count"].transform("sum")

grouped_stats["percentage"] = (grouped_stats["count"] / grouped_stats["total_count"]) * 100

stats_pivot = grouped_stats.pivot_table(
    index=["topic", "platform", "annotator"],
    columns="bias",
    values=["count", "percentage"],
    fill_value=0
).reset_index()

In [ ]:
grouped_stats

,topic,platform,annotator,bias,count,total_count,percentage
0,AI,Reddit,bs,affect_h,11,37,29.729730
1,AI,Reddit,bs,all_or_nothing,20,37,54.054054
2,AI,Reddit,bs,representativeness_h,6,37,16.216216
3,AI,Reddit,fs,affect_h,1199,1579,75.934136
4,AI,Reddit,fs,all_or_nothing,31,1579,1.963268
...,...,...,...,...,...,...,...
65,climate,Twitter,wl,affect_h,50,128,39.062500
66,climate,Twitter,wl,all_or_nothing,7,128,5.468750
67,climate,Twitter,wl,availability_h,14,128,10.937500
68,climate,Twitter,wl,confirmation_b,48,128,37.500000


In [ ]:
stats_pivot

topic platform annotator    count                                \
bias                             affect_h all_or_nothing availability_h   
0          AI   Reddit        bs     11.0           20.0            0.0   
1          AI   Reddit        fs   1199.0           31.0          165.0   
2          AI   Reddit        ir      0.0           49.0            0.0   
3          AI   Reddit        wl     32.0           20.0            9.0   
4          AI  Twitter        bs      1.0            0.0            1.0   
5          AI  Twitter        fs    107.0            0.0           10.0   
6          AI  Twitter        ir     22.0            0.0           20.0   
7          AI  Twitter        wl      8.0            0.0            1.0   
8     climate   Reddit        bs     21.0           12.0           23.0   
9     climate   Reddit        fs    628.0           18.0          170.0   
10    climate   Reddit        ir     83.0           36.0          102.0   
11    climate   Reddit        wl     39.0           19.0           15.0   
12    climate  Twitter        bs      1.0            3.0            1.0   
13    climate  Twitter        fs    425.0            3.0           38.0   
14    climate  Twitter        ir     35.0           26.0           47.0   
15    climate  Twitter        wl     50.0            7.0           14.0   

                                         percentage                 \
bias confirmation_b representativeness_h   affect_h all_or_nothing   
0               0.0                  6.0  29.729730      54.054054   
1             153.0                 31.0  75.934136       1.963268   
2              76.0                  0.0   0.000000      39.200000   
3               9.0                 14.0  38.095238      23.809524   
4               0.0                  1.0  33.333333       0.000000   
5              30.0                  6.0  69.934641       0.000000   
6               4.0                  5.0  43.137255       0.000000   
7               4.0                  1.0  57.142857       0.000000   
8              53.0                  2.0  18.918919      10.810811   
9             434.0                 21.0  49.409913       1.416208   
10             87.0                106.0  20.048309       8.695652   
11             19.0                 12.0  37.500000      18.269231   
12             20.0                  1.0   3.846154      11.538462   
13             65.0                 15.0  77.838828       0.549451   
14             28.0                 35.0  20.467836      15.204678   
15             48.0                  9.0  39.062500       5.468750   

                                                         
bias availability_h confirmation_b representativeness_h  
0          0.000000       0.000000            16.216216  
1         10.449652       9.689677             1.963268  
2          0.000000      60.800000             0.000000  
3         10.714286      10.714286            16.666667  
4         33.333333       0.000000            33.333333  
5          6.535948      19.607843             3.921569  
6         39.215686       7.843137             9.803922  
7          7.142857      28.571429             7.142857  
8         20.720721      47.747748             1.801802  
9         13.375295      34.146341             1.652242  
10        24.637681      21.014493            25.603865  
11        14.423077      18.269231            11.538462  
12         3.846154      76.923077             3.846154  
13         6.959707      11.904762             2.747253  
14        27.485380      16.374269            20.467836  
15        10.937500      37.500000             7.031250

In [ ]:
bias_columns = [f"{base}_{annotator}" for annotator in annotators for base in bias_bases]

df_long = whole_dataset.melt(
    id_vars=["topic", "platform"],
    value_vars=bias_columns,
    var_name="bias_annotator",
    value_name="value"
)

df_long[['bias', 'annotator']] = df_long["bias_annotator"].str.rsplit('_', n=1, expand=True)
df_long_filtered = df_long[df_long["value"] == 1]

agg = (
    df_long_filtered
    .groupby(['annotator', 'bias'])
    .size()
    .reset_index(name='Count')
)

agg["Percent of annotator's total"] = (
    agg["Count"]
    / agg.groupby("annotator")["Count"].transform("sum")
    * 100
).round(2)

agg = agg.rename(columns={'annotator': 'Annotator', 'bias': 'Bias'})

In [ ]:
agg[['Annotator','Bias','Count',"Percent of annotator's total"]]

,Annotator,Bias,Count,Percent of annotator's total
0,bs,affect_h,34,19.21
1,bs,all_or_nothing,35,19.77
2,bs,availability_h,25,14.12
3,bs,confirmation_b,73,41.24
4,bs,representativeness_h,10,5.65
5,fs,affect_h,2359,66.47
6,fs,all_or_nothing,52,1.47
7,fs,availability_h,383,10.79
8,fs,confirmation_b,682,19.22
9,fs,representativeness_h,73,2.06


# IAA metrics
Each topic x platform file and the Main_Agree_All_HAnn were iterated through the IAA below.

## Fleiss Kappa


In [ ]:
review_df = r2_ai.copy()

### Bias presence

In [ ]:
bias_presence_df = review_df.copy()
bias_columns = [col for col in bias_presence_df.columns if col not in ['topic']]
bias_presence_df[bias_columns] = bias_presence_df[bias_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

for col in bias_columns:
    bias_presence_df[col] = np.where(bias_presence_df[col] > 0, 1, 0)

bias_counts = bias_presence_df[bias_columns].apply(lambda row: np.bincount(row, minlength=2), axis=1)
bias_counts_df = pd.DataFrame(bias_counts.tolist(), columns=['no_bias', 'bias'])

kappa = fleiss_kappa(bias_counts_df.values, method='fleiss')
print(f"Fleiss' Kappa for bias/no bias agreement: {kappa:.4f}")

Fleiss' Kappa for bias/no bias agreement: 0.0177


### Each bias

In [ ]:
fleiss_kappa_results = {}

for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']:
    annotator_columns = [f"{bias}_{annotator}" for annotator in ['bs', 'fs', 'ir', 'wl'] if f"{bias}_{annotator}" in bias_presence_df.columns]

    bias_counts = bias_presence_df[annotator_columns].apply(lambda row: np.bincount(row, minlength=2), axis=1)
    bias_counts_df = pd.DataFrame(bias_counts.tolist(), columns=['no_bias', 'bias'])

    if bias_counts_df['no_bias'].sum() == 0 or bias_counts_df['bias'].sum() == 0:
        print(f"Insufficient variability for {bias}. Assigning Fleiss' Kappa: 0.0000")
        fleiss_kappa_results[bias] = 0.0
        continue

    kappa = fleiss_kappa(bias_counts_df.values, method='fleiss')
    fleiss_kappa_results[bias] = kappa
    print(f"Fleiss' Kappa for {bias}: {kappa:.4f}")

Fleiss' Kappa for affect_h: 0.0357
Fleiss' Kappa for availability_h: 0.0089
Fleiss' Kappa for representativeness_h: 0.0220
Fleiss' Kappa for confirmation_b: 0.0181
Fleiss' Kappa for all_or_nothing: 0.0610


## Cohen's Kappa

### Bias presence

In [ ]:
annotators = ['bs', 'fs', 'ir', 'wl']
kappa_results_bias_no_bias = {}

annotator_pairs = list(itertools.combinations(annotators, 2))
for annotator1, annotator2 in annotator_pairs:
    col1 = [f"{bias}_{annotator1}" for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']]
    col2 = [f"{bias}_{annotator2}" for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']]

    combined_col1 = bias_presence_df[col1].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    combined_col2 = bias_presence_df[col2].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

    kappa = cohen_kappa_score(combined_col1, combined_col2)
    kappa_results_bias_no_bias[(annotator1, annotator2)] = kappa
    print(f"Cohen's Kappa for Bias/No Bias between {annotator1} and {annotator2}: {kappa:.4f}")

Cohen's Kappa for Bias/No Bias between bs and fs: 0.0965
Cohen's Kappa for Bias/No Bias between bs and ir: 0.0667
Cohen's Kappa for Bias/No Bias between bs and wl: 0.1498
Cohen's Kappa for Bias/No Bias between fs and ir: 0.2061
Cohen's Kappa for Bias/No Bias between fs and wl: 0.0479
Cohen's Kappa for Bias/No Bias between ir and wl: 0.0409


### Each bias

In [ ]:
kappa_results = {}

def clean_column(column):
    return pd.to_numeric(column.replace(r"[^\d]", "", regex=True), errors='coerce').fillna(0).astype(int)

for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']:
    annotator_pairs = list(itertools.combinations(annotators, 2))

    kappa_scores = {}
    for annotator1, annotator2 in annotator_pairs:
        col1 = f"{bias}_{annotator1}"
        col2 = f"{bias}_{annotator2}"

        if col1 in review_df.columns and col2 in review_df.columns:
            col1_values = clean_column(review_df[col1])
            col2_values = clean_column(review_df[col2])

            if col1_values.nunique() < 2 or col2_values.nunique() < 2:
                kappa = 0.0
                print(f"Insufficient unique labels for {bias} between {annotator1} and {annotator2}. Assigned Kappa: {kappa:.4f}")
            else:
                kappa = cohen_kappa_score(col1_values, col2_values)
                if pd.isna(kappa):
                    kappa = 0.0

            kappa_scores[(annotator1, annotator2)] = kappa
            print(f"Cohen's Kappa for {bias} between {annotator1} and {annotator2}: {kappa:.4f}")

    kappa_results[bias] = kappa_scores

print("\nDetailed Cohen's Kappa results:")
for bias, scores in kappa_results.items():
    print(f"\nBias Category: {bias}")
    for pair, kappa in scores.items():
        annotator1, annotator2 = pair
        print(f"  {annotator1} vs {annotator2}: {kappa:.4f}")

Cohen's Kappa for affect_h between bs and fs: 0.0473
Cohen's Kappa for affect_h between bs and ir: 0.0435
Cohen's Kappa for affect_h between bs and wl: 0.2150
Cohen's Kappa for affect_h between fs and ir: 0.1055
Cohen's Kappa for affect_h between fs and wl: 0.0603
Cohen's Kappa for affect_h between ir and wl: 0.0390
Cohen's Kappa for availability_h between bs and fs: 0.0332
Cohen's Kappa for availability_h between bs and ir: -0.0014
Cohen's Kappa for availability_h between bs and wl: -0.0007
Cohen's Kappa for availability_h between fs and ir: 0.0139
Cohen's Kappa for availability_h between fs and wl: -0.0014
Cohen's Kappa for availability_h between ir and wl: 0.0332
Cohen's Kappa for representativeness_h between bs and fs: -0.0055
Cohen's Kappa for representativeness_h between bs and ir: 0.0482
Insufficient unique labels for representativeness_h between bs and wl. Assigned Kappa: 0.0000
Cohen's Kappa for representativeness_h between bs and wl: 0.0000
Cohen's Kappa for representativenes

## Weighted F1 Score

### Any bias

In [ ]:
f1_results_bias_no_bias = {}

annotator_pairs = list(itertools.combinations(annotators, 2))
for annotator1, annotator2 in annotator_pairs:
    col1 = [f"{bias}_{annotator1}" for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']]
    col2 = [f"{bias}_{annotator2}" for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']]

    col1 = [c for c in col1 if c in bias_presence_df.columns]
    col2 = [c for c in col2 if c in bias_presence_df.columns]

    if col1 and col2:
        combined_col1 = bias_presence_df[col1].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
        combined_col2 = bias_presence_df[col2].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

        f1 = f1_score(combined_col1, combined_col2, average='weighted')
        f1_results_bias_no_bias[(annotator1, annotator2)] = f1
        print(f"F1 Score for Bias/No Bias between {annotator1} and {annotator2}: {f1:.4f}")

F1 Score for Bias/No Bias between bs and fs: 0.8243
F1 Score for Bias/No Bias between bs and ir: 0.8782
F1 Score for Bias/No Bias between bs and wl: 0.9622
F1 Score for Bias/No Bias between fs and ir: 0.6894
F1 Score for Bias/No Bias between fs and wl: 0.6196
F1 Score for Bias/No Bias between ir and wl: 0.7487


### Each bias

In [ ]:
f1_results_per_bias = {}
for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']:
    f1_scores = {}
    for annotator1, annotator2 in annotator_pairs:
        col1 = f"{bias}_{annotator1}"
        col2 = f"{bias}_{annotator2}"

        if col1 in bias_presence_df.columns and col2 in bias_presence_df.columns:
            f1 = f1_score(bias_presence_df[col1], bias_presence_df[col2], average='weighted')
            f1_scores[(annotator1, annotator2)] = f1
            print(f"F1 Score for {bias} between {annotator1} and {annotator2}: {f1:.4f}")

    f1_results_per_bias[bias] = f1_scores

F1 Score for affect_h between bs and fs: 0.8639
F1 Score for affect_h between bs and ir: 0.9456
F1 Score for affect_h between bs and wl: 0.9863
F1 Score for affect_h between fs and ir: 0.7026
F1 Score for affect_h between fs and wl: 0.6859
F1 Score for affect_h between ir and wl: 0.8741
F1 Score for availability_h between bs and fs: 0.9797
F1 Score for availability_h between bs and ir: 0.9789
F1 Score for availability_h between bs and wl: 0.9986
F1 Score for availability_h between fs and ir: 0.9256
F1 Score for availability_h between fs and wl: 0.9411
F1 Score for availability_h between ir and wl: 0.9432
F1 Score for representativeness_h between bs and fs: 0.9897
F1 Score for representativeness_h between bs and ir: 0.9836
F1 Score for representativeness_h between bs and wl: 0.9928
F1 Score for representativeness_h between fs and ir: 0.9816
F1 Score for representativeness_h between fs and wl: 0.9907
F1 Score for representativeness_h between ir and wl: 0.9701
F1 Score for confirmation_b 

## Gwet's AC1

### Bias presence

In [ ]:
gwet_results_bias_no_bias = {}

bias_no_bias_cols = [
    f"{bias}_{annotator}" for bias in ['affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']
    for annotator in annotators if f"{bias}_{annotator}" in bias_presence_df.columns
]

if len(bias_no_bias_cols) >= 2:
    df_bias_no_bias = bias_presence_df[bias_no_bias_cols].copy()
    df_bias_no_bias.columns = [col.split('_')[-1] for col in bias_no_bias_cols]

    cac_instance = CAC(df_bias_no_bias)
    gwet_result = cac_instance.gwet()
    gwet_results_bias_no_bias['bias_no_bias'] = gwet_result

    print("Gwet's AC1 for Bias/No Bias:")
    print({
        'est': {
            'coefficient_value': gwet_result['est']['coefficient_value'],
            'coefficient_name': gwet_result['est']['coefficient_name'],
            'confidence_interval': gwet_result['est']['confidence_interval'],
            'p_value': gwet_result['est']['p_value'],
            'z': gwet_result['est']['z'],
            'se': gwet_result['est']['se'],
            'pa': gwet_result['est']['pa'],
            'pe': gwet_result['est']['pe']
        },
        'weights': gwet_result['weights'],
        'categories': gwet_result['categories']
    })


Gwet's AC1 for Bias/No Bias:
{'est': {'coefficient_value': 0.93877, 'coefficient_name': 'AC1', 'confidence_interval': (0.93378, 0.94375), 'p_value': 0.0, 'z': 369.36706, 'se': 0.00254, 'pa': 0.9424, 'pe': 0.05926}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}


#### For pairs

In [ ]:
gwet_results_pairs_bias_no_bias = {}
annotator_pairs = list(itertools.combinations(annotators, 2))

bias_no_bias_cols = [
    f"{bias}_{annotator}" for bias in biases
    for annotator in annotators if f"{bias}_{annotator}" in bias_presence_df.columns
]

for annotator1, annotator2 in annotator_pairs:
    col1 = [f"{bias}_{annotator1}" for bias in biases if f"{bias}_{annotator1}" in bias_presence_df.columns]
    col2 = [f"{bias}_{annotator2}" for bias in biases if f"{bias}_{annotator2}" in bias_presence_df.columns]

    if col1 and col2:
        combined_col1 = bias_presence_df[col1].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
        combined_col2 = bias_presence_df[col2].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

        df_pair = pd.DataFrame({
            annotator1: combined_col1,
            annotator2: combined_col2
        })

        cac_instance = CAC(df_pair)
        gwet_result = cac_instance.gwet()
        gwet_results_pairs_bias_no_bias[(annotator1, annotator2)] = gwet_result

        print(f"Gwet's AC1 for Bias/No Bias between {annotator1} and {annotator2}:")
        print({
            'est': {
                'coefficient_value': gwet_result['est']['coefficient_value'],
                'coefficient_name': gwet_result['est']['coefficient_name'],
                'confidence_interval': gwet_result['est']['confidence_interval'],
                'p_value': gwet_result['est']['p_value'],
                'z': gwet_result['est']['z'],
                'se': gwet_result['est']['se'],
                'pa': gwet_result['est']['pa'],
                'pe': gwet_result['est']['pe']
            },
            'weights': gwet_result['weights'],
            'categories': gwet_result['categories']
        })

Gwet's AC1 for Bias/No Bias between bs and fs:
{'est': {'coefficient_value': 0.63765, 'coefficient_name': 'AC1', 'confidence_interval': (0.59915, 0.67615), 'p_value': 0.0, 'z': 32.48947, 'se': 0.01963, 'pa': 0.7326, 'pe': 0.26204}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for Bias/No Bias between bs and ir:
{'est': {'coefficient_value': 0.77526, 'coefficient_name': 'AC1', 'confidence_interval': (0.74648, 0.80404), 'p_value': 0.0, 'z': 52.84485, 'se': 0.01467, 'pa': 0.81668, 'pe': 0.18429}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for Bias/No Bias between bs and wl:
{'est': {'coefficient_value': 0.96409, 'coefficient_name': 'AC1', 'confidence_interval': (0.95396, 0.97422), 'p_value': 0.0, 'z': 186.68686, 'se': 0.00516, 'pa': 0.96554, 'pe': 0.0405}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for Bias/No Bias between fs and ir:
{'est': {'coefficient_value': 0.55173, 'coeffici

### Each bias

In [ ]:
gwet_results = {}

for bias in biases:
    annotator_cols = [f"{bias}_{annotator}" for annotator in annotators if f"{bias}_{annotator}" in bias_presence_df.columns]

    if len(annotator_cols) >= 2:
        df_bias = bias_presence_df[annotator_cols].copy()
        df_bias.columns = [col.split('_')[-1] for col in annotator_cols]

        unique_categories = pd.concat([df_bias[col] for col in df_bias.columns]).nunique()
        if unique_categories < 2:
            print(f"Only one unique category for {bias}. Assigning Gwet's AC1: 1.0000")
            gwet_results[bias] = {
                'est': {'coefficient_value': 1.0, 'coefficient_name': 'AC1', 'confidence_interval': (1.0, 1.0), 'p_value': 0.0, 'z': float('nan'), 'se': float('nan'), 'pa': 1.0, 'pe': 1.0},
                'weights': None,
                'categories': [df_bias.iloc[0, 0]]
            }
            continue

        cac_instance = CAC(df_bias)
        gwet_result = cac_instance.gwet()
        gwet_results[bias] = gwet_result

        print(f"Gwet's AC1 for {bias}:")
        print({
            'est': {
                'coefficient_value': gwet_result['est']['coefficient_value'],
                'coefficient_name': gwet_result['est']['coefficient_name'],
                'confidence_interval': gwet_result['est']['confidence_interval'],
                'p_value': gwet_result['est']['p_value'],
                'z': gwet_result['est']['z'],
                'se': gwet_result['est']['se'],
                'pa': gwet_result['est']['pa'],
                'pe': gwet_result['est']['pe']
            },
            'weights': gwet_result['weights'],
            'categories': gwet_result['categories']
        })


Gwet's AC1 for affect_h:
{'est': {'coefficient_value': 0.82251, 'coefficient_name': 'AC1', 'confidence_interval': (0.80547, 0.83956), 'p_value': 0.0, 'z': 94.65537, 'se': 0.00869, 'pa': 0.8501, 'pe': 0.15545}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for availability_h:
{'est': {'coefficient_value': 0.95959, 'coefficient_name': 'AC1', 'confidence_interval': (0.95205, 0.96714), 'p_value': 0.0, 'z': 249.55474, 'se': 0.00385, 'pa': 0.96118, 'pe': 0.03917}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for representativeness_h:
{'est': {'coefficient_value': 0.98472, 'coefficient_name': 'AC1', 'confidence_interval': (0.98013, 0.9893), 'p_value': 0.0, 'z': 421.14374, 'se': 0.00234, 'pa': 0.98495, 'pe': 0.01539}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for confirmation_b:
{'est': {'coefficient_value': 0.94723, 'coefficient_name': 'AC1', 'confidence_interval': (0.9385, 0.95595), 'p_

#### For pairs

In [ ]:
gwet_results = {}
annotator_pairs = [(annotators[i], annotators[j]) for i in range(len(annotators)) for j in range(i + 1, len(annotators))]

for bias in biases:
    pair_results = {}
    for annotator1, annotator2 in annotator_pairs:
        col1 = f"{bias}_{annotator1}"
        col2 = f"{bias}_{annotator2}"

        if col1 in bias_presence_df.columns and col2 in bias_presence_df.columns:
            pair_df = bias_presence_df[[col1, col2]].copy()
            pair_df.columns = [annotator1, annotator2]

            if pair_df[annotator1].nunique() < 2 or pair_df[annotator2].nunique() < 2:
                print(f"Insufficient unique labels for {bias} between {annotator1} and {annotator2}. Assigning AC1: 0.0000")
                pair_results[(annotator1, annotator2)] = 0.0
                continue

            cac_instance = CAC(pair_df)
            gwet_result = cac_instance.gwet()
            pair_results[(annotator1, annotator2)] = gwet_result

            print(f"Gwet's AC1 for {bias} between {annotator1} and {annotator2}:")
            print({
                'est': {
                    'coefficient_value': gwet_result['est']['coefficient_value'],
                    'coefficient_name': gwet_result['est']['coefficient_name'],
                    'confidence_interval': gwet_result['est']['confidence_interval'],
                    'p_value': gwet_result['est']['p_value'],
                    'z': gwet_result['est']['z'],
                    'se': gwet_result['est']['se'],
                    'pa': gwet_result['est']['pa'],
                    'pe': gwet_result['est']['pe']
                },
                'weights': gwet_result['weights'],
                'categories': gwet_result['categories']
            })

    gwet_results[bias] = pair_results

Gwet's AC1 for affect_h between bs and fs:
{'est': {'coefficient_value': 0.71096, 'coefficient_name': 'AC1', 'confidence_interval': (0.67753, 0.74438), 'p_value': 0.0, 'z': 41.72269, 'se': 0.01704, 'pa': 0.77257, 'pe': 0.21317}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for affect_h between bs and ir:
{'est': {'coefficient_value': 0.90148, 'coefficient_name': 'AC1', 'confidence_interval': (0.88386, 0.9191), 'p_value': 0.0, 'z': 100.36458, 'se': 0.00898, 'pa': 0.91041, 'pe': 0.09058}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for affect_h between bs and wl:
{'est': {'coefficient_value': 0.98526, 'coefficient_name': 'AC1', 'confidence_interval': (0.9789, 0.99161), 'p_value': 0.0, 'z': 303.95051, 'se': 0.00324, 'pa': 0.98553, 'pe': 0.01843}, 'weights': array([[1., 0.],
       [0., 1.]]), 'categories': [0, 1]}
Gwet's AC1 for affect_h between fs and ir:
{'est': {'coefficient_value': 0.65655, 'coefficient_name': 'AC1